Install Needed Packages - geocoder and Beautiful Soup

In [1]:
#conda install -c conda-forge geocoder

In [2]:
 #conda install -c anaconda beautifulsoup4 

Import Packages

In [3]:
import requests
import bs4 as bs
from bs4 import BeautifulSoup

In [5]:
import pandas as pd
import numpy as np
import geocoder

Scrape Wikipedia for Toronto Postal Codes

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

TorPostCodes = requests.get(url).text
#print(TorPostCodes)

In [4]:
TPC = BeautifulSoup(TorPostCodes, 'html.parser')

Parse Scraped Wikipedia Page into dataframe

In [6]:
table = TPC.find('table')
#print(TPC)
#print(table)
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


df = pd.DataFrame(res, columns=["PostCode", "Bourough", "Neighborhood"])
df.head()

,PostCode,Bourough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove 'Not Assigned' Bouroughs and Neigborhoods

In [7]:
df = df[df.Bourough != 'Not assigned']
df.head()

,PostCode,Bourough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Combine Neigborhoods by Post Code

In [8]:
df = df.groupby(['PostCode','Bourough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
df.head(10)

,PostCode,Bourough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [9]:
df["Neighborhood"] = np.where(df["Neighborhood"]!='Not assigned',df["Neighborhood"],df["Bourough"])

In [10]:
df.head()

,PostCode,Bourough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Add Coordinate Fields to dataframe

In [11]:
df["Latitude"] = 0
df["Longitude"] = 0

Obtain City Coordinates for Toronto

In [12]:
t = geocoder.arcgis('Toronto, Ontario')
lat_lng_coords = t.latlng
TorLat = lat_lng_coords[0]
TorLong = lat_lng_coords[1]

Obtain Coordinates by Post Code

In [13]:
for i, row in df.iterrows():
    search = df["PostCode"].loc[i] + ', Toronto, Ontario'
    g = geocoder.arcgis(search)
    lat_lng_coords = g.latlng
    df["Latitude"].loc[i] = lat_lng_coords[0]
    df["Longitude"].loc[i] = lat_lng_coords[1]
df.head(10)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,PostCode,Bourough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696665,-79.260163


Import Libraries for Plotting, API Calls, Clustring

In [14]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

Create Toronto Map with Neigborhoods

In [15]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[TorLat, TorLong], zoom_start=10)

# add markers to map
for Latitude, Longitude, Bourough, Neighborhood in zip(df['Latitude'], df['Longitude'], df['Bourough'], df['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Bourough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

Set Up Foursquare API Call

In [16]:
CLIENT_ID = 'KW0MTDFH1TMS5ML3GOKPAJHL0CCHGNKHNJ3UL3GVDVL2RYZC' # your Foursquare ID
CLIENT_SECRET = 'C2O3A0PD2U2KG01MFVYF3BJRL4OL1B1IYO0OAHDS3BJHHIO5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KW0MTDFH1TMS5ML3GOKPAJHL0CCHGNKHNJ3UL3GVDVL2RYZC
CLIENT_SECRET:C2O3A0PD2U2KG01MFVYF3BJRL4OL1B1IYO0OAHDS3BJHHIO5


In [17]:
LIMIT = 100
radius = 500
Version = '20180605'

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, TorLat, TorLong, VERSION, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=KW0MTDFH1TMS5ML3GOKPAJHL0CCHGNKHNJ3UL3GVDVL2RYZC&client_secret=C2O3A0PD2U2KG01MFVYF3BJRL4OL1B1IYO0OAHDS3BJHHIO5&ll=43.648690000000045,-79.38543999999996&v=20180605&radius=500&limit=100'

Define and call Function to Explore each neighborhood

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
TorVenues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes = df['Latitude'],
                                   longitudes = df['Longitude']
                                  )
TorVenues

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.811525,-79.195517,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
2,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood,Morningside,West Hill",43.765690,-79.175256,The Strawberry Patch,43.764738,-79.173081,Tea Room
5,"Guildwood,Morningside,West Hill",43.765690,-79.175256,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
6,"Guildwood,Morningside,West Hill",43.765690,-79.175256,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center
7,"Guildwood,Morningside,West Hill",43.765690,-79.175256,Heron Park,43.769327,-79.177201,Park
8,Woburn,43.768359,-79.217590,Starbucks,43.770037,-79.221156,Coffee Shop
9,Woburn,43.768359,-79.217590,cheapOseo,43.766042,-79.218539,Business Service


Rename Venue Category that is a Duplicate of "Neighborhood" that would cause conflict with column name

In [20]:
TorVenues["Venue Category"] = np.where(TorVenues["Venue Category"]!='Neighborhood',TorVenues["Venue Category"],"Neigborhood Services")

One Hot Encoding by Venue Category

In [21]:
# one hot encoding
TorOneHot = pd.get_dummies(TorVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
TorOneHot['Neighborhood'] = TorVenues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [TorOneHot.columns[-1]] + list(TorOneHot.columns[:-1])
TorOneHot = TorOneHot[fixed_columns]
TorOneHot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neigborhood Services,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [22]:
TorGrouped = TorOneHot.groupby('Neighborhood').mean().reset_index()
TorGrouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neigborhood Services,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.03,0.0,0.0,0.01,0.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,0.01,0.03,0.0,0.0,0.0,0.01,0.000000,0.0,0.0,0.0,0.02,0.0,0.01,0.02,0.0,0.0,0.000000,0.0,0.01,0.03,0.01,0.0,0.0,0.0,0.0,0.06,0.0,0.0,0.0,0.0,0.0,0.000000,

Explore Top Venues by Neighborhood

In [23]:
num_top_venues = 5

for hood in TorGrouped['Neighborhood']:
    print("----"+hood+"----")
    temp = TorGrouped[TorGrouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0              Café  0.06
1       Coffee Shop  0.06
2             Hotel  0.05
3               Bar  0.03
4  Asian Restaurant  0.03


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.14
1       Shopping Mall  0.14
2                Pool  0.07
3      Discount Store  0.07
4              Bakery  0.07


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                   Pharmacy   1.0
1             Adult Boutique   0.0
2    New American Restaurant   0.0
3         Mexican Restaurant   0.0
4  Middle Eastern Restaurant   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
           venue  freq
0  Grocery Store  0.13
1    Pizza Place  0.07
2           Park  0.07
3   Liquor Store  0.07
4    Coffee Shop  0.07


----Alderwood,Long Branch----
                   venue  freq
0         Sandwich Place   0.2
1     

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = TorGrouped['Neighborhood']

for ind in np.arange(TorGrouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TorGrouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Burger Joint,American Restaurant,Bakery,Restaurant,Deli / Bodega,Steakhouse,Asian Restaurant
1,Agincourt,Chinese Restaurant,Shopping Mall,Pool,Shanghai Restaurant,Supermarket,Sushi Restaurant,Bakery,Discount Store,Department Store,Bubble Tea Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Pharmacy,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Japanese Restaurant,Park,Coffee Shop,Sandwich Place,Fast Food Restaurant,Liquor Store,Beer Store,Fried Chicken Joint
4,"Alderwood,Long Branch",Candy Store,Performing Arts Venue,Pub,Sandwich Place,Gym,Yoga Studio,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


Cluster Neighborhoods by frequency of Venue Categories

In [26]:
# set number of clusters
kclusters = 5
TorGroupedClustering = TorGrouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters).fit(TorGroupedClustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 0, 4, 0, 0, 4, 0], dtype=int32)

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

TorMerged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
TorMerged = TorMerged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

TorMerged.head() # check the last columns!

,PostCode,Bourough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517,1.0,Home Service,Food,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750,0.0,Construction & Landscaping,Moving Target,Bar,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256,4.0,Tea Room,Park,Gym / Fitness Center,Construction & Landscaping,Cuban Restaurant,Cupcake Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.768359,-79.217590,2.0,Business Service,Park,Coffee Shop,Korean Restaurant,Field,Fast Food Restaurant,Fish & Chips Shop,Farmers Market,Eastern European Restaurant,Farm
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,4.0,Playground,Trail,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Dog Run


In [28]:
TorMerged = TorMerged[np.isfinite(TorMerged['Cluster Labels'])]

TorMerged["Cluster Labels"] = TorMerged["Cluster Labels"].round(0).astype(int)

TorMerged.head(10)

,PostCode,Bourough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517,1,Home Service,Food,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750,0,Construction & Landscaping,Moving Target,Bar,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256,4,Tea Room,Park,Gym / Fitness Center,Construction & Landscaping,Cuban Restaurant,Cupcake Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.768359,-79.217590,2,Business Service,Park,Coffee Shop,Korean Restaurant,Field,Fast Food Restaurant,Fish & Chips Shop,Farmers Market,Eastern European Restaurant,Farm
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,4,Playground,Trail,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Dog Run
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750,0,Train Station,Restaurant,Grocery Store,Indian Restaurant,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726245,-79.263670,0,Chinese Restaurant,Discount Store,Coffee Shop,Department Store,Falafel Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Yoga Studio
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713133,-79.285055,0,Bakery,Bus Line,Coffee Shop,Intersection,Soccer Field,Metro Station,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976,0,Fast Food Restaurant,Pizza Place,Pharmacy,Burger Joint,Furniture / Home Store,Discount Store,Sandwich Place,Liquor Store,Coffee Shop,Ethiopian Restaurant
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696665,-79.260163,4,Gym Pool,College Stadium,Skating Rink,Gym,Park,General Entertainment,Farm,Farmers Market,Falafel Restaurant,Donut Shop


Visualize Neighborhood Clusters

In [29]:
# create map
TorMapClust = folium.Map(location=[TorLat, TorLong], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TorMerged['Latitude'], TorMerged['Longitude'], TorMerged['Neighborhood'], TorMerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(TorMapClust)
       
TorMapClust

Display Neighborhoods by Cluster

In [30]:
TorMerged.loc[TorMerged['Cluster Labels'] == 0, TorMerged.columns[[2] + list(range(5, TorMerged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek,Rouge Hill,Port Union",0,Construction & Landscaping,Moving Target,Bar,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
5,Scarborough Village,0,Train Station,Restaurant,Grocery Store,Indian Restaurant,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
6,"East Birchmount Park,Ionview,Kennedy Park",0,Chinese Restaurant,Discount Store,Coffee Shop,Department Store,Falafel Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Yoga Studio
7,"Clairlea,Golden Mile,Oakridge",0,Bakery,Bus Line,Coffee Shop,Intersection,Soccer Field,Metro Station,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
8,"Cliffcrest,Cliffside,Scarborough Village West",0,Fast Food Restaurant,Pizza Place,Pharmacy,Burger Joint,Furniture / Home Store,Discount Store,Sandwich Place,Liquor Store,Coffee Shop,Ethiopian Restaurant
10,"Dorset Park,Scarborough Town Centre,Wexford He...",0,Bakery,Gift Shop,Brewery,Light Rail Station,Falafel Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Farm
11,"Maryvale,Wexford",0,Intersection,Auto Garage,Farm,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Yoga Studio,Dumpling Restaurant
12,Agincourt,0,Chinese Restaurant,Shopping Mall,Pool,Shanghai Restaurant,Supermarket,Sushi Restaurant,Bakery,Discount Store,Department Store,Bubble Tea Shop
13,"Clarks Corners,Sullivan,Tam O'Shanter",0,Pharmacy,Hobby Shop,Bus Stop,Thai Restaurant,Coffee Shop,Fried Chicken Joint,Shopping Mall,Pizza Place,Chinese Restaurant,Farm
15,L'Amoreaux West,0,Fast Food Restaurant,Chinese Restaurant,Electronics Store,Pizza Place,Pharmacy,American Restaurant,Sandwich Place,Other Great Outdoors,Coffee Shop,Grocery Store


In [31]:
TorMerged.loc[TorMerged['Cluster Labels'] == 1, TorMerged.columns[[2] + list(range(5, TorMerged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",1,Home Service,Food,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
63,Roselawn,1,Home Service,Food,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [32]:
TorMerged.loc[TorMerged['Cluster Labels'] == 2, TorMerged.columns[[2] + list(range(5, TorMerged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Woburn,2,Business Service,Park,Coffee Shop,Korean Restaurant,Field,Fast Food Restaurant,Fish & Chips Shop,Farmers Market,Eastern European Restaurant,Farm
30,"CFB Toronto,Downsview East",2,Airport,Food Court,Coffee Shop,Park,Yoga Studio,Falafel Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit
37,The Beaches,2,Health Food Store,Coffee Shop,Pub,Neigborhood Services,Yoga Studio,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
49,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",2,Coffee Shop,Light Rail Station,Convenience Store,Supermarket,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Eastern European Restaurant,Farm
85,Queen's Park,2,Coffee Shop,Café,Sandwich Place,Bubble Tea Shop,Indian Restaurant,Italian Restaurant,Theater,Salad Place,Sushi Restaurant,Juice Bar
97,"Emery,Humberlea",2,Coffee Shop,Nightclub,Park,Yoga Studio,Exhibit,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant


In [33]:
TorMerged.loc[TorMerged['Cluster Labels'] == 3, TorMerged.columns[[2] + list(range(5, TorMerged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North,L'Amoreaux East,Milliken,Steel...",3,Pharmacy,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
64,"Forest Hill North,Forest Hill West",3,Pharmacy,Park,Yoga Studio,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Farmers Market


In [34]:
TorMerged.loc[TorMerged['Cluster Labels'] == 4, TorMerged.columns[[2] + list(range(5, TorMerged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood,Morningside,West Hill",4,Tea Room,Park,Gym / Fitness Center,Construction & Landscaping,Cuban Restaurant,Cupcake Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
4,Cedarbrae,4,Playground,Trail,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Dog Run
9,"Birch Cliff,Cliffside West",4,Gym Pool,College Stadium,Skating Rink,Gym,Park,General Entertainment,Farm,Farmers Market,Falafel Restaurant,Donut Shop
19,Bayview Village,4,Dog Run,Park,Trail,Construction & Landscaping,Cuban Restaurant,Cupcake Shop,Flea Market,Fish Market,Fish & Chips Shop,Field
23,York Mills West,4,Speakeasy,Korean Restaurant,Park,Bank,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit
25,Parkwoods,4,Park,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Yoga Studio
26,Don Mills North,4,Burger Joint,Bus Line,Park,Intersection,Yoga Studio,Farm,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
31,Downsview West,4,Park,Mobile Phone Shop,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
34,Victoria Village,4,Intersection,Grocery Store,Park,French Restaurant,Farm,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
40,East Toronto,4,Bar,Park,Farmers Market,Italian Restaurant,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit
